<a href="https://colab.research.google.com/github/akash748/SentimentAnalysisUsingBert/blob/main/Bert_Implementation_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#Importing the libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
#Instantiate the model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

In [15]:
#Encode and Calculate Sentiment
tokens = tokenizer.encode('The movie was bad but the actors were good',return_tensors='pt')

In [16]:
tokens

tensor([[  101, 10103, 13113, 10140, 12428, 10502, 10103, 26826, 10342, 12050,
           102]])

In [17]:
tokenizer.decode(tokens[0])

'[CLS] the movie was bad but the actors were good [SEP]'

In [18]:
result = model(tokens)

In [20]:
result.logits

tensor([[ 0.2909,  1.2525,  1.3958, -0.2161, -2.2398]],
       grad_fn=<AddmmBackward>)

In [21]:
int(torch.argmax(result.logits))+1

3

In [23]:
#Collect Reviews
r = requests.get('https://www.yelp.com/biz/dumpling-baby-china-bistro-san-francisco-4')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class' : regex})
reviews = [i.text for i in results]
reviews

['Just had dinner from here. Ordered chicken kung pao, shrimp dumplings, and steam chicken bao buns. Delicious! The shrimp dumplings are larger and fatter then your standard. The filling for the chicken bao was not overly sweet with soft fluffy buns. The portion of the Kung Pao was large. Price is reasonable considering the current inflation on everything these days. Would definitely return to try other items!',
 'I was craving some noodles and dumplings and I was very much satisfied with my order!For the Shanghai Chubby Noodles, 5 stars. I added chicken to my order and it was so well prepared, good flavor and very juicy!Shanghai Wontons in chili oil were 4 stars. Maybe 4.5 but what is brining my rating down is that I was expecting them to be more flavorful in being mixed well with the oil, but I did not get that at all. Top layer was "dry" or rather plain because all the oil was at the very bottom of the container. Appears to be that they serve the oil first then add the wontons on to

In [25]:
#Load Reviews into DataFrame
df = pd.DataFrame(np.array(reviews),columns = ['reviews'])

In [26]:
df.head()

,reviews
0,Just had dinner from here. Ordered chicken kun...
1,I was craving some noodles and dumplings and I...
2,This got one star bc we ate it.I have never we...
3,"I wish I could say only good things, but I wou..."
4,Nice place in the neighborhood to order some j...


In [27]:
def sentiment_score(review):
  tokens = tokenizer.encode(review,return_tensors='pt')
  result = model(tokens)
  return (int(torch.argmax(result.logits))+1)

In [31]:
sentiment_score(df['reviews'].iloc[1])

4

In [33]:
df.shape

(10, 1)

In [35]:
df['score'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [37]:
df['score'].mean()

3.4

The restaurant has a average rating of 3.4 !!